
# Unstructured Grids

MODFLOW6 supports unstructured grids. Unlike raster data, the connectivity of
unstructured grids is irregular. This means that the number of neighboring
cells is not constant; for structured grids, every cell has 4 neighbors (in
2D), or 6 neighbors (in 3D).
